# Dynamics of the system

In [3]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, vlatex
import numpy as np
from IPython.display import display, Math
from phaseportrait import PhasePortrait2D

# The equations

<img src="assets/system_image.jpg" alt="alt text" width="600"/>

## Variables & symbolic constants  

Constants:
* $m_d, m_B, m_E, m_{BE}, m_{AD}$ -- masses of disk, connector, end-effector and rods $BE$ and $AD$ respectively
* $R, l_{BE}, l_{AD}$ -- radius of the disk, length of $BE$ and $AD$ respectively
* $I_e$ -- inertia of end-effector around center of mass
* Coordinates of point $A$
* Angle of rotation of gravity force $\theta_g$ 

In [41]:
m_d, m_B, m_E, m_BE, m_AD = sp.symbols('m_d, m_B, m_E, m_{BE}, m_{AD}')
m_rod = m_B + m_E + m_BE + m_AD
R, l_BE, l_AD = sp.symbols('R, l_{BE}, l_{AD}')
I_e = sp.symbols('I_e')
x_A, y_A = sp.symbols('x_A, y_A')
r_A = sp.Matrix([x_A, y_A])
theta_g = sp.symbols('theta_g')
g = sp.symbols('g')

I_d = m_d * R**2 / 2

Variables:
* $\theta(t)$ -- generalized coordinate, angle of rotations of the disk
* $\tau_{ext}$ -- external torque, applied to the disk

In [42]:
t = sp.symbols('t')
theta, tau_ext = dynamicsymbols('theta, tau_{ext}') 
dtheta = theta.diff(t)

## System's points as function of coordinate

There are many points & parameters in the system, which is important to derive from the system coordinates. These are:
* $r_B, r_M, r_F, r_E$ -- coordinates of points $B, M, F, E$ respectively
* Angle $\alpha$ between rod and horizon
* $l_{com}$ is a specific parameter, representing distance between point B and the center of mass of the inner rod
* $\frac{r_B - r_a}{||r_B - r_A||}$, represented as `r_AB_norm` is a normal vector, collinear with the rod

In [47]:
x_B, y_B = R * sp.cos(theta), R * sp.sin(theta)
r_B = sp.Matrix([x_B, y_B])

alpha = sp.atan2(y_B - y_A, x_B - x_A)
r_AB_norm = sp.Matrix([sp.cos(alpha), sp.sin(alpha)])

r_M = r_A + r_AB_norm * l_AD / 2
r_F = r_B + r_AB_norm * l_BE / 2
r_E = r_B + r_AB_norm * l_BE

l_com = (r_B - r_A).norm() - l_AD / 2

In [48]:
l_com.diff(t)

(((-re(x_A) + re(R*cos(theta(t))))*Derivative(re(R*cos(theta(t))), t) + (-im(x_A) + im(R*cos(theta(t))))*Derivative(im(R*cos(theta(t))), t))*Abs(R*cos(theta(t)) - x_A)*sign(R*cos(theta(t)) - x_A)/(R*cos(theta(t)) - x_A) + ((-re(y_A) + re(R*sin(theta(t))))*Derivative(re(R*sin(theta(t))), t) + (-im(y_A) + im(R*sin(theta(t))))*Derivative(im(R*sin(theta(t))), t))*Abs(R*sin(theta(t)) - y_A)*sign(R*sin(theta(t)) - y_A)/(R*sin(theta(t)) - y_A))/sqrt(Abs(R*sin(theta(t)) - y_A)**2 + Abs(R*cos(theta(t)) - x_A)**2)

It is also quite handy to have an angular velocity $\dot{\alpha}$ to calculate kinetic energy of the system. Moreover, the variable $K$ is introduced, such that:
$$\dot{\alpha} = K_\alpha \dot{\theta}$$

In [5]:
dalpha = sp.diff(alpha, t)
K_alpha = sp.diff(alpha, theta)

display(Math(f'$\dot{{\\alpha}} = {vlatex(dalpha)} = K \dot{{\\theta}}$'))

<IPython.core.display.Math object>

## Energy of the system

### Velocity of rod's center of mass

One of the key component of the kinetic energy of the system is the velocity of center of mass. 

The position itself could be expressed as follows:
$$r_{com} = \frac{1}{m_{rod}} (m_{AD} r_M + m_B r_B + m_{BE} r_F + m_E r_E)$$

Despite the `sympy` is cabable of differentiating the velocity symbolically, it is useful to derive these equations manually:
$$v_{com} = \frac{1}{m_{rod}}\left( (m_{AD} \frac{l_{AD}}{2} + m_{BE} \frac{l_{BE}}{2} + m_E l_{BE}) \begin{bmatrix} -\sin \alpha \\ \cos \alpha \end{bmatrix} K_\alpha +  (m_B + m_{BE} + m_E) \begin{bmatrix} -\sin \theta \\ \cos \theta \end{bmatrix}\right) \dot{\theta} = D \dot{\theta}$$

In [6]:
r_com = (m_AD * r_M + m_B * r_B + m_BE * r_F + m_E * r_E) / m_rod
D = ((m_AD * l_AD / 2 + m_BE * l_BE / 2 + m_E * l_BE) * sp.Matrix([-sp.sin(alpha), sp.cos(alpha)]) * K_alpha + (m_B + m_BE + m_E) * r_AB_norm.diff(theta)) / m_rod

### Inertia of the rod

The intertia of the rod around rotational point $B$ is as follows:
$$I_r = \frac{m_{AD}}{3} (\frac{l_{AD}^2}{4} + 3 l_{com}^2) + \frac{1}{3} m_{BE} l_{BE}^2 + I_E + m_E l_{BE}^2$$

In [7]:
I_r = m_AD / 3 * (l_AD ** 2 / 4 + 3 * l_com**2) + m_BE * l_BE**2 / 3 + I_e + m_E * l_BE ** 2

### Kinetic Energy

Using all calculations above, the kinetic energy could be formulated as follows:
$$K = \frac{1}{2} m_{rod} v_{com}^T v_{com} + \frac{1}{2} I_d \dot{\theta}^2 + \frac{1}{2} I_r \dot{\alpha}^2 = \frac{1}{2} M \dot{\theta}^2,$$
where:
$$M = m_{rod} D^T D  + I_d + I_r K^2_\alpha$$

In [8]:
M = m_rod * D.norm()**2 + I_d + I_r * K_alpha**2
M

R**2*m_d/2 + (-R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))**2*(I_e + l_{BE}**2*m_E + l_{BE}**2*m_{BE}/3 + m_{AD}*(l_{AD}**2/4 + 3*(-l_{AD}/2 + sqrt(Abs(R*sin(theta(t)) - y_A)**2 + Abs(R*cos(theta(t)) - x_A)**2))**2)/3) + (Abs((-(R*sin(theta(t)) - y_A)*(R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))*(l_{AD}*m_{AD}/2 + l_{BE}*m_E + l_{BE}*m_{BE}/2)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + (R*sin(theta(t))/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - (R*cos(theta(t)) - x_A)*(-R*(R*sin(theta(t)) - y_A)*cos(theta(t)) + R*(R*cos(theta(t)) - x_A)*sin(theta(t)))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)**(3/2))*(m_B + m_E + m_{BE}))/

### Potential energy

The only source of potential energy in the system is a gravitational force, affecting . Hence, the equation for a potential energy is:
$$P = m_{rod} g y_{com} \cos \theta_g$$

In [9]:
P = m_rod * g * r_com[1] * sp.cos(theta_g)
P

g*(R*m_B*sin(theta(t)) + m_E*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + m_{AD}*(l_{AD}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + y_A) + m_{BE}*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))))*cos(theta_g)

## Resulting equation

Given values of $M$, $P$, and external torque law $\tau_{ext}$, one might formulate an equation:
$$M \ddot{\theta} + \dot{M} \dot{\theta} - \frac{1}{2} \frac{d}{d\theta} (M\dot{\theta}^2) + \frac{dP}{d\theta} = \tau_{ext}$$ 

By expanding the term $\frac{1}{2} \frac{d}{d\theta} (M\dot{\theta}^2) = \frac{1}{2}\dot{M}\dot{\theta}$, obtain:
$$M \ddot{\theta} + \frac{1}{2} \dot{M} \dot{\theta} + \frac{dP}{d\theta} = \tau_{ext}$$

In [10]:
M_dot = sp.diff(M, t)
dP_dtheta = sp.diff(P, theta)

In [11]:
ddtheta = (-M_dot * dtheta/2 + dP_dtheta - tau_ext) / M

# Numerical computing

Unfortunately, due to high complexity of the resulting dynamics, `sp.lambdify()` fails to treat them in reasonable time. Hence, the function for $\ddot{\theta}$ has to be written manually.

## Constant initialization

In [3]:
m_d, m_B, m_E, m_BE, m_AD = 0.1, 0.4, 0.6, 0.08, 0.05
m_rod = m_B + m_E + m_BE + m_AD
R, l_BE, l_AD = .12, .2, .3
I_e = 0.5
x_A, y_A = -0.09, -0.11
r_A = sp.Matrix([x_A, y_A])
theta_g = 0
g = 9.81

I_d = m_d * R**2 / 2

In [21]:
class ModelDynamics():
    def __init__(
            self, 
            m_d=.1,         # Mass of the disk
            R=.12,          # Radius of the disk
            m_B=.4,         # Mass of connector B
            m_E=.6,         # Mass of end-effector E
            I_e=.5,         # Moment of inertia for end-effector E aroud CoM
            m_BE=.08,       # Mass of outer rod BE
            l_BE=.2,        # Length of outer rod BE
            m_AD=.05,       # Mass of inner rod AD
            l_AD=.3,        # length of inner rod AD
            x_A=-0.09,      # x coordinate of point A
            y_A=-0.11,      # y coordinate of point A
            theta_g=0       # Angle of gravitational force
        ):
        # Constant terms
        self.m_d, self.m_B, self.m_E, self.m_BE, self.m_AD = m_d, m_B, m_E, m_BE, m_AD
        self.R, self.l_BE, self.l_AD = R, l_BE, l_AD
        self.I_e = I_e
        self.r_A = np.array([x_A, y_A])
        self.theta_g = theta_g
        self.g = 9.81

        # Error message to print when the system state is not provided
        self.no_input_error_text = "The system state is not provided"

        # System state
        self.theta = None
        self.theta_dot = None
        self.tau = None

        self.update_constant_terms()
        self.reset_internal_params()


    def reset_internal_params(self):
        # Reset all the internal parameters to indefined state.
        # It should be called each time constant value or system
        # state is reassigned 
         
        self.r_B = None
        self.r_B_dot = None
        self.alpha = None
        self.rod_norm = None
        self.l_AB = None
        self.r_M = None
        self.l_com = None
        self.l_com_dot = None
        self.r_F = None
        self.r_E = None
        self.r_com = None
        
        self.K_alpha = None
        self.K_alpha_dot = None
        self.D = None
        self.D_dot = None
        self.I_r = None
        self.I_r_dot = None
        self.M = None
        self.M_dot = None
        self.P = None
        self.dP_dtheta = None
        self.energy = None
        self.theta_ddot = None


    def update_constant_terms(self):
        # Update constant terms, which are based on the initial 
        # system parameters. It should be called each time a 
        # constant is reassigned.

        self.m_rod = self.m_B + self.m_E + self.m_BE + self.m_AD
        
        self.I_d = self.m_d * self.R**2 / 2
        
        self.D_norm = (self.m_AD * self.l_AD / 2 + self.m_BE * self.l_BE / 2 + self.m_E * self.l_BE) / self.m_rod
        self.D_theta = (self.m_B + self.m_BE + self.m_E) / self.m_rod

    # TODO: write setter for r_A, g_theta, theta, theta_dot, and tau 
    #       they should call methods update_constant_terms (for first two) 
    #       and reset_internal_params

    def set_system_state(self, theta, theta_dot, tau=0):
        # Sets the system state with a given values

        self.theta = theta
        self.theta_dot = theta_dot
        self.tau = tau

        # After the values are changes, the internal
        # parameters need to be resetted
        self.reset_internal_params()
    

    def _eval_r_B(self):
        # Evaluates point's B coordinates
        #
        #       ┌ R * cos(theta) ┐
        # r_B = |                |   
        #       └ R * sin(theta) ┘
         
        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)

        x_B = self.R * np.cos(self.theta)
        y_B = self.R * np.sin(self.theta)

        self.r_B = np.array([x_B, y_B])


    def _eval_r_B_dot(self):
        # Evaluates time derivative of r_B
        #
        #           ┌ -R * sin(theta) ┐ 
        # r_B_dot = |                 | theta_dot
        #           └  R * cos(theta) ┘ 

        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)

        x_B_dot = -self.R * np.sin(self.theta) * self.theta_dot
        y_B_dot = self.R * np.cos(self.theta) * self.theta_dot

        self.r_B_dot = np.array([x_B_dot, y_B_dot])


    def _eval_alpha(self):
        # Evaluates alpha -- angle between the ground and 
        # the rod
        #
        # alpha = atan2(y_B - y_A, x_B - x_A)

        if self.r_B is None:
            self._eval_r_B()

        self.alpha = np.arctan2(self.r_B[1] - self.r_A[1], self.r_B[0] - self.r_A[0])


    def _eval_rod_norm(self):
        # Evaluates unit vector, pointing in the direction
        # of the rod
        #
        #             ┌ cos(alpha) ┐
        # rod_norm  = |            |   
        #             └ sin(alpha) ┘

        if self.alpha is None:
            self._eval_alpha()
        self.rod_norm = np.array([np.cos(self.alpha), np.sin(self.alpha)])


    def _eval_l_AB(self):
        # Evaluates distance between points A and B
        #
        # l_AB = ||r_B - r_A||_2
    
        if self.r_B is None:
            self._eval_r_B()

        self.l_AB = np.linalg.norm(self.r_B - self.r_A)

    
    def _eval_r_M(self):
        # Evaluates coordinate of poitn M
        #                      l_AD
        # r_M = r_A + rod_norm ----
        #                       2

        if self.alpha is None:
            self._eval_alpha()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_M = self.r_A + self.rod_norm * self.l_AD / 2

    
    def _eval_l_com(self):
        # Evaluates l_com -- distance betwee CoM of the rod
        # and the point B
        #                 1
        # l_com = l_AB - --- l_AD
        #                 2

        if self.l_AB is None:
            self._eval_l_AB()
        if self.r_B is None:
            self._eval_r_B()
        
        self.l_com = self.l_AB - self.l_AD / 2
    

    def _eval_l_com_dot(self):
        # Evaluates time derivative of l_com
        #              (x_B - x_A + y_B - y_A)(x_B_dot + y_B_dot)
        # l_com_dot = --------------------------------------------  
        #                                l_AB

        if self.l_AB is None:
            self._eval_l_AB()
        if self.r_B is None:
            self._eval_r_B()
        if self.r_B_dot is None:
            self._eval_r_B_dot()
        
        self.l_com_dot = (self.r_B[0] - self.r_A[0] + self.r_B[1] - self.r_A[1]) * (self.r_B_dot[0] + self.r_B_dot[1]) / self.l_AB
    

    def _eval_r_F(self):
        # Evaluates coordinate of point F -- CoM of the outher rod
        #                       l_BE
        # r_F = r_B + rod_norm  ----
        #                        2

        if self.r_B is None:
            self._eval_r_B()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_F = self.r_B + self.rod_norm * self.l_BE/2
    

    def _eval_r_E(self):
        # Evaluates coordinate of point E -- CoM of end-effector
        # 
        # r_E = r_B + rod_norm * l_BE
        
        if self.r_B is None:
            self._eval_r_B()
        if self.rod_norm is None:
            self._eval_rod_norm()
        
        self.r_E = self.r_B + self.rod_norm * self.l_BE
    

    def _eval_r_com(self):
        # Evaluates coordinate of CoM of the rod
        #          m_AD * r_A          
        # r_com = ------------ + D_norm * rod_norm + D_theta * r_B 
        #            m_rod          

        if self.rod_norm is None:
            self._eval_rod_norm()
        if self.r_B is None:
            self._eval_r_B()

        self.r_com = self.m_AD * self.r_A / self.m_rod + \
            self.D_norm * self.rod_norm + \
                self.D_theta * self.r_B


    def _eval_K_alpha(self):
        # Evaluate K_alpha, which is defined as alpha_dot = K_alpha theta_dot
        #            R^2 - x_A * R cos(theta) - y_a * R sin(theta)
        # K_alpha = -----------------------------------------------
        #                                l_AB^2

        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)
        if self.l_AB is None:
            self._eval_l_AB()
        if self.r_B is None:
            self._eval_r_B()

        Rcos = self.R * np.cos(self.theta)
        Rsin = self.R * np.sin(self.theta)

        self.K_alpha = (self.R**2 - self.r_B[0]*Rcos - self.r_B[1]*Rsin) / self.l_AB**2


    def _eval_K_alpha_dot(self):
        # TODO: check this derivative
        #
        # Evaluate time derivative of K_alpha
        #                -R^2 * x_A * sin(theta) + R^2 * y_A * cos(theta) + x_A^3 * sin(theta) -
        # K_alpha_dot = -----------------------------------------------------------------------------
        #                                                        l_AB**4
        #
        #     - x_A^2 * y_A * cos(theta) + x_A * y_A^2 * sin(theta) - y_A^3 * cos(theta)
        #   ------------------------------------------------------------------------------ R * theta_dot
        #                                           l_AB**4

        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        if self.r_B is None:
            self._eval_r_B()

        sin_theta = np.sin(self.theta)
        cos_theta = np.cos(self.theta)

        x_A, y_A = self.r_A

        self.K_alpha_dot = self.R * (-self.R**2 * x_A * sin_theta + \
            self.R**2 * y_A * cos_theta + \
                x_A**3 * sin_theta - \
                    x_A**2 * y_A * cos_theta + \
                        x_A * y_A**2 * sin_theta - \
                            y_A**3 * cos_theta) / self.l_AB**4 * self.theta_dot


    def _eval_D(self):
        # Evaluate D, which is defined as v_com = D * theta_dot
        #            ┌ -sin(alpha) ┐                   ┌ -sin(theta) ┐
        # D = D_norm |             | K_alpha + D_theta |             |
        #            └  cos(alpha) ┘                   └  cos(theta) ┘

        if self.theta is None:
            raise RuntimeError(self.no_input_error_text)
        if self.alpha is None:
            self._eval_alpha()
        if self.K_alpha is None:
            self._eval_K_alpha()

        self.D = self.D_norm * np.array([-np.sin(self.alpha), np.cos(self.theta)]) * self.K_alpha + \
            self.D_theta * np.array([-np.sin(self.theta), np.cos(self.theta)]) 
    

    def _eval_D_dot(self):
        # Evaluate time derivative of D
        #            ┌ -sin(alpha) ┐               ┌ -cos(alpha) ┐                      
        # D = D_norm |             | K_alpha_dot + |             | K_alpha^2 theta_dot +
        #            └  cos(alpha) ┘               └ -sin(alpha) ┘                      
        #
        #            ┌ -cos(theta) ┐
        #   + D_theta|             | theta_dot
        #            └ -sin(theta) ┘

        if self.theta is None or self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        if self.K_alpha is None:
            self._eval_K_alpha()
        if self.K_alpha_dot is None:
            self._eval_K_alpha_dot()
        if self.alpha is None:
            self._eval_alpha()

        self.D_dot = self.D_norm * np.array([-np.sin(self.alpha), np.cos(self.alpha)]) * self.K_alpha_dot + \
            self.D_norm * np.array([-np.cos(self.alpha), -np.sin(self.alpha)]) * self.K_alpha**2 * self.theta_dot + \
            self.D_theta * np.array([-np.cos(self.theta), -np.sin(self.theta)]) * self.theta_dot


    def _eval_I_r(self):
        # Evaluate inertia of the rod around point B
        #       m_AD ╭ 1                    ╮    1
        # I_r = ---- |--- l_AD^2 + 3 l_com^2| + --- m_BE l_BE^2 + I_E + m_E l_BE^2
        #        3   ╰ 4                    ╯    3

        if self.l_com is None:
            self._eval_l_com()
        
        self.I_r = self.m_AD * (self.l_AD ** 2 / 4 + 3 * self.l_com**2) / 3 + \
            self.m_BE * self.l_BE ** 2 / 3 + self.I_e + self.m_E * self.l_BE**2


    def _eval_I_r_dot(self):
        # Evaluate time derivative of I_r
        #
        # I_r_dot = 2 m_AD * l_com * l_com_dot
        
        if self.l_com is None:
            self._eval_l_com()
        if self.l_com_dot is None:
            self._eval_l_com_dot()
        
        self.I_r_dot = 2 * self.m_AD * self.l_com * self.l_com_dot


    def _eval_M(self):
        # TODO: check this formula
        #
        # Evaluate generalized mass M
        #
        # M = m_rod * D^T D + I_d + I_r * K_alpha^2
        
        if self.D is None:
            self._eval_D()
        if self.I_r is None:
            self._eval_I_r()
        if self.K_alpha is None:
            self._eval_K_alpha()

        self.M = self.m_rod * self.D.T @ self.D + self.I_d + self.I_r * self.K_alpha ** 2


    def _eval_M_dot(self):
        # Evaluate time derivative of M
        #
        # M_dot = 2 m_rod * D^T D_dot + I_r_dot * K_alpha**2 + 2 I_r * K_alpha * K_alpha_dot 
        #
        if self.D is None:
            self._eval_D()
        if self.D_dot is None:
            self._eval_D_dot()
        if self.I_r is None:
            self._eval_I_r()
        if self.I_r_dot is None:
            self._eval_I_r_dot()
        if self.K_alpha is None:
            self._eval_K_alpha()
        if self.K_alpha_dot is None:
            self._eval_K_alpha_dot()

        self.M_dot =  2 * self.m_rod * self.D.T @ self.D_dot + self.I_r_dot * self.K_alpha**2 + 2 * self.I_r * self.K_alpha * self.K_alpha_dot 


    def _eval_P(self):
        # Evaluates potential energy
        #
        # P = m_rod * g * y_com * cos(theta_g)

        if self.r_com is None:
            self._eval_r_com()

        self.P = self.m_rod * self.g * self.r_com[1] * np.cos(self.theta_g)
    
    
    def _eval_dP_dtheta(self):
        # Evaluates derivative of P w.r.t. theta
        #
        # dP_dtheta = m_rod * g * D_y * cos(theta_g)

        if self.D is None:
            self._eval_D()

        self.dP_dtheta = self.m_rod * self.g * self.D[1] * np.cos(self.theta_g)

    def _eval_energy(self):
        # Evaluates energy of the system
        #      1
        # E = --- M * D^T D * theta_dot^2 + P
        #      2

        if self.theta_dot is None:
            raise RuntimeError(self.no_input_error_text)
        if self.M is None:
            self._eval_M()
        if self.D is None:
            self._eval_D()
        if self.P is None:
            self._eval_P()
        
        self.energy = self.M * self.D.T @ self.D * self.theta_dot**2 / 2 + self.P


    def _eval_theta_ddot(self):
        # Evaluate double time derivative of theta
        #               1  ╭   1                                     ╮
        # theta_ddot = --- |- --- M_dot * theta_dot - dP_dtheta + tau|
        #               M  ╰   2                                     ╯

        if self.theta_dot is None or self.tau is None:
            raise RuntimeError(self.no_input_error_text)
        if self.M_dot is None:
            self._eval_M_dot()
        if self.M is None:
            self._eval_M()
        if self.dP_dtheta is None:
            self._eval_dP_dtheta()

        self.theta_ddot = (-self.M_dot * self.theta_dot / 2 - self.dP_dtheta + self.tau) / self.M


    def get_theta_ddot(self, params = None):
        # (kinda) Getter for theta_ddot. Allows to pass 
        # the parameters not to initialize the system separately.

        if not params is None:
            if len(params) < 2 or len(params) > 3:
                raise RuntimeError('Invalid parameters of the system')
            if len(params) == 2:
                self.set_system_state(params[0], params[1])
            else:
                self.set_system_state(params[0], params[1], params[2])
        if self.theta_ddot is None:
            self._eval_theta_ddot()

        return self.theta_ddot

    def get_energy(self, params = None):
        # (kinda) Getter for energy. Allows to pass
        # the parameters not to initialize the system separately
        if not params is None:
            if len(params) < 2 or len(params) > 3:
                raise RuntimeError('Invalid parameters of the system')
            if len(params) == 2:
                self.set_system_state(params[0], params[1])
            else:
                self.set_system_state(params[0], params[1], params[2])
        if self.energy is None:
            self._eval_energy()
        
        return self.energy

In [22]:
dynamics = ModelDynamics()
dynamics.set_system_state(1, 2)
dynamics.get_theta_ddot()

-5.540676662772198

In [23]:
dynamics = ModelDynamics()
E = dynamics.get_energy((1, 2, 3))
E

3.9745873241802454